# Applying functions to data files in batch

This package contains functionality for defining a Python function and applying it to remote files in batches using the `gsbatch_apply` class. This notebook gives an example of how to use this class.

In the getting_started notebook, we looked at creating batch object. These objects describe a list of files (local or remote on google buckets), separating them into batches and then downloading or uploading the data. Here, we extend that to show how we can easily perform a workflow with the following steps:

    1. Download data from a google bucket in batches.
    2. Apply a function to this batch of data one file at a time.
    3. Save the analysed data to new files.
    4. Push the resulting batch of new files back to the google bucket.
    
The `gsbatch_apply` object works by performing the following loop on any batch objects that are passed:

1. Download data using `gsutil -m cp` from any batch objects where `source='remote'`. Data is downloaded to directory argument `get_dir` (required). 
2. Pass files from the current batch of all objects to a user provided function in the same order that batches were passed to gsbatch_apply.
3. Function can be applied to batch files one at a time (`pass_args='one'`) or all at once (`pass_args = 'all'`).
4. Concatenate any output from all function applications in the batch.
5. Upload data using `gsutil -m cp` to remote directory `put_dir` (required if there are objects with `source='local'`). 
6. Delete temporary downloaded files from this batch.
7. If `delete_put_files == True` then delete any batch files described by any objects where `source = 'local'`.
8. Cycle all objects up to the next batch.
9. Go back to step one until all batches are cycled.

So let's start by importing things:

In [41]:
%load_ext autoreload
%autoreload 2
from gsbatch import Gsbatch, gsbatch_apply
import xarray as xr
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Next, we're going to create two Gsbatch objects. The first describes the data in the google bucket. The second describes local data that does not exist yet (but will once we have analysed bucket data).

In [71]:
# Create a batch object describing things on the cloud
gsb_get = Gsbatch(gs_dir = "gs://fqqzlp/test_data",
                  gs_list = '*.nc',
                  source='remote',
                  batch_size=4,
                  get_dir = "/home/davidbyrne/disks/ssd500/test")

# Create local batch object with same number of files and batch size
# as the remote batch object. These will be names test0.nc, ..., testN.nc
output_filenames = [f"test{ii}.nc" for ii in range(gsb_get.n_files)]
gsb_put = Gsbatch(gs_dir = "/home/davidbyrne/disks/ssd500/test",
                  gs_list = output_filenames,
                  source='local',
                  batch_size=4,
                  put_dir = "gs://fqqzlp/test_data")

Next we need to define a function to apply. Here we are going to take two file names `fp_in` and `fp_out`. We will open `fp_in` and take the mean of the data along a single axis (`'x'`) and save it to `fp_out`. We're using `xarray` to do this here, so that it can be done over multiple cores in parallel.

We also return an integer (1) when the routine has been successful just to show how gsbatch_apply will concatenate outputs.

In [83]:
def batch_func(fp_in, fp_out):
    ds = xr.open_dataset(fp_in, chunks={'lat':1e3, 'lon':1e3})
    data_out = ds.mean(dim='lat')
    data_out.to_netcdf(fp_out)
    return 1

Now we call `gsbatch_apply`:

In [87]:
%%time
app = gsbatch_apply(func = batch_func, 
                    batch = [gsb_get, gsb_put],
                    verbosity = 2,
                    pass_args = 'one',
                    delete_put_files = True)

  Applying function batch_func to 2 gsbatch object.

   --> Resetting all batches
   --> Number of batches: 3
   --> Processing batch: 1 / 3
      --> Getting data from 1 gsbatch objects.
      --> Applying function one file at a time.
      --> Uploading data to 1 gsbatch objects.
   --> Processing batch: 2 / 3
      --> Getting data from 1 gsbatch objects.
      --> Applying function one file at a time.
      --> Uploading data to 1 gsbatch objects.
   --> Processing batch: 3 / 3
      --> Getting data from 1 gsbatch objects.
      --> Applying function one file at a time.
      --> Uploading data to 1 gsbatch objects.
Final batch has been reached. Cannot increase index.
Final batch has been reached. Cannot increase index.
Done! Phew.
CPU times: user 51.5 s, sys: 1min 5s, total: 1min 56s
Wall time: 5min 17s


In [85]:
app.output

[1, 1, 1, 1, 1]